In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [ ]:
class BMTradingRule():
    # strategy: always long top 10% by B/M and short bottom 10% by B/M, equal-weighted or value-weighted
    # For each $1 NAV, we open $1 total of long positions AND $1 total of short positions
    # This would be the max leverage allowed given 50% margin requirements
    #
    # Assume 100% of portfolio liquidated each month and repurchased with new quantities
    
    # strategy-specifc columns for trades_df (variables we want to keep track of for subsequent analysis)
    # usually the variables that led the strategy to open trade in the first place
    # In this case, just book-to-market ratio 'bm'
    strategy_specific_trades_df_columns = {'bm': pd.Series([], dtype='float')} 
    
    # minumum number of stocks with available bm for us to actually trade
    min_stocks_available = 100
    
    # value weighted?
    value_weighted = False
    
    def __init__(self,portfolio_db):
        self.portfolio_db = portfolio_db
    
    # Regardless of the strategy you are implementing, this method must return
    # open_trades_df, close_trades_df
    #
    # open_trades_df is a DataFrame with all the required trades_df columns plus any custom ones for this strategy
    #     each row is a new trade the strategy wants to open
    #     method only populates the security_id (index) and quantity required columns, plus any custom columns. Rest remain NaN to be populated elsewhere
    # close_trades_df is a DataFrame that is a subset of the rows of portfolio_db.trades_df
    #     each row is an exist trade the strategy wants to close
    #     we don't need to populate any columns in this function
    def compute_trades(self,signal_df):                
        # Since we are doing 100% turnover each period, all currently open trades should be closed
        close_trades_df = self.portfolio_db.trades_df.loc[ self.portfolio_db.trades_df.loc[:,'close_datetime'].isna() ,: ].copy()
        
        # before deciding to open new positions, check that signal_df gives us a big enough set of stocks to actually do this
        if( len(signal_df) < self.min_stocks_available ):
            return self.empty_trades_df(), close_trades_df       
        
        # To compute open_trades, we need bm, which we define as book equity (seqq) divided by market cap (cshoq*prccq)
        signal_df.loc[:,'bm'] = signal_df.loc[:,'seqq'] / (signal_df.loc[:,'cshoq']*signal_df.loc[:,'prccq'])
        signal_col_name = 'bm'
        
        # You shouldn't have to change anything below here if you are doing a decile strategy    
            
        # Find top 10% of signal column      
        # need to make a copy because we are going to modify this and don't want to affect signal_df
        buys_df = signal_df.loc[ signal_df.loc[:,signal_col_name] > signal_df.loc[:,signal_col_name].quantile(0.9)].copy() 
        
        # Find bottom 10% of signal column
        # need to make a copy because we are going to modify this and don't want to affect signal_df
        sells_df = signal_df.loc[ signal_df.loc[:,signal_col_name] < signal_df.loc[:,signal_col_name].quantile(0.1)].copy() 
    
        if( self.value_weighted ): 
            # Compute value weights
            buys_mktcap = buys_df.loc[:,'prc']*buys_df.loc[:,'shrout']
            buys_w = buys_mktcap / buys_mktcap.sum()
            sells_mktcap = sells_df.loc[:,'prc']*sells_df.loc[:,'shrout']
            sells_w = -sells_mktcap / sells_mktcap.sum() # weights negative because we are selling
            
            # Dollars per position equal total NAV * buys_w and NAV * sells_w, respectively
            buys_dollar = self.portfolio_db.current_nav() * buys_w
            buys_df.loc[:,'quantity'] = buys_dollar / buys_df.loc[:,'prc']
            sells_dollar = self.portfolio_db.current_nav() * sells_w
            sells_df.loc[:,'quantity'] = sells_dollar / sells_df.loc[:,'prc']             
                        
        else:
            # With equal weights we can go directly to dollars per buy/sell
            # Which is just NAV / number of positions
            dollars_per_buy = self.portfolio_db.current_nav() / len(buys_df)
            buys_df['quantity'] = dollars_per_buy / buys_df['prc']
      
            # figure out quantity per trade by splitting $ evenly
            dollars_per_sell = -self.portfolio_db.current_nav() / len(sells_df) # negative because we are selling
            sells_df['quantity'] = dollars_per_sell / sells_df['prc'] 
        
        # now make open_trades_df, keeping only security_id, quantity, and signal_col_name from the buys_df and sells_df
        
        # Append is going to be deprecated, so I'm relacing this
        # open_trades_df = self.empty_trades_df().append(
        #     buys_df.loc[:,['security_id','quantity',signal_col_name]]).append(
        #     sells_df.loc[:,['security_id','quantity',signal_col_name]]) 
        
        # With this
        open_trades_df = pd.concat([self.empty_trades_df(), buys_df.loc[:,['security_id','quantity',signal_col_name]], sells_df.loc[:,['security_id','quantity',signal_col_name]] ])
        
        return open_trades_df, close_trades_df
    
    # Returns an empty trades_df
    # Used so we know the right columns to populate when creating a trades_df else
    def empty_trades_df(self):
        return pd.concat([self.portfolio_db.empty_trades_df(), pd.DataFrame(self.strategy_specific_trades_df_columns)], axis=1)
        